In [12]:
import contextily
import sklearn
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from pyecharts.charts import Bar
from pyecharts.charts import Boxplot
from pyecharts import options as opts
import pandas as pd
from owslib.wfs import WebFeatureService
import os
from pyecharts import Bar
from pyecharts import options as opts

In [13]:
dest = "data"
docs = "docs"

In [14]:
# funzione che scarica i dati e produce file geojson e file csv
def updatedata(dest="data"):
    wfs_url="http://sit.comune.bolzano.it/geoserver/wfs?service=WFS"
    wfs = WebFeatureService(url=wfs_url, version='2.0.0')
    layers = list(wfs.contents)
    incidenti = []
    for layer in layers:
        if layer.find('ciden') > -1:
            incidenti.append(layer)
    for incidente in incidenti:
        response = wfs.getfeature(typename=incidente)
        data = gpd.read_file(response)
        data.crs="epsg:25832"
        data = data.to_crs(epsg=4326)
        data['DTINCID']=data['DTINCID'].astype(str)
        data['ANNOINC']= = data['ANNOINC'].astype(int)
        data.DTINCID =data.DTINCID.apply(lambda x: x.replace(' 00:00:00+00:00',""))
        data['lon'] = data.geometry.x
        data['lat'] = data.geometry.y
        name = incidente.replace("Cartografia:view_","")
        data.to_file(dest + os.sep + name + ".geojson",driver="GeoJSON")
        del data['geometry']
        data.to_csv(dest + os.sep + name + ".csv",index=False)

In [15]:
download = False
if (download):
    updatedata()

In [25]:
incidenti_prod = gpd.read_file("data" + os.sep + "incidenti_prod.geojson")

In [26]:
incidenti_prod.fillna("informazione non disponibile",inplace=True)

In [59]:
incidenti_prod.ANNOINC.astype(int)

0        2021
1        2021
2        2021
3        2021
4        2021
         ... 
11982    2022
11983    2022
11984    2022
11985    2022
11986    2022
Name: ANNOINC, Length: 11987, dtype: int64

In [39]:
incidenti_natura = incidenti_prod.groupby(['NATURA']).size().reset_index().rename(columns={0:'total'}).sort_values(['total','NATURA'],ascending=[False,True])

In [49]:
incidenti_natura.total.to_list()

[7769, 950, 806, 675, 374, 348, 286, 273, 254, 104, 67, 36, 34, 10, 1]

In [57]:


(Bar()
    .add_xaxis(incidenti_natura.NATURA.to_list())
    .add_yaxis('Natura incidente', incidenti_natura.total.to_list())
    .set_global_opts(title_opts=opts.TitleOpts(title="Totale infortuni stradali per natura", subtitle="2013-oggi"))
    .render("docs/index.html") # generate a local HTML file
)

'/home/napo/dev/incidenti_bolzano/docs/index.html'